In [22]:
import os
import sys
import re
import json
from dotenv import load_dotenv
import random
from docling.document_converter import DocumentConverter

In [3]:
load_dotenv()

True

In [4]:
dir_path='./reviews'
list_of_files=os.listdir(dir_path)

In [5]:
len(list_of_files)

55

In [20]:
def shuffle(data):
    answer=random.choice(['A', 'B', 'C', 'D'])
    if(answer!=data['correct_option']):
        string1=data['options'][answer]
        string2=data['options'][data['correct_option']]
        data['options'][data['correct_option']]=string1
        data['options'][answer]=string2
        data['correct_option']=answer
    return data

In [24]:
from openai import OpenAI
client = OpenAI()


prompt='''for the paper below, please create 15 questions with 4 answeres one of which is correct. 
An example is shown below. Please ground the paper in the content, however do not refer to figures or numbers, 
design more qualitative questions. Give a reference to the text. Outline the topic and the difficulty.

Example of QA pair:
{ "topic": "activated_esters",
   "difficulty": "easy",
   "question": "Why are activated esters particularly attractive for polymer functionalization?",
   "options": {
        "A": "They eliminate the need for solvents",
          "B": "They are stable toward radical polymerization and react under mild conditions",
          "C": "They only react with aromatic amines",
          "D": "They prevent hydrolysis completely."
        },
    "correct_option": "B",
    "answer": "They are stable toward radical polymerization and react under mild conditions",
    "justification": {
        "anchor": "Activated ester advantages",
        "rationale": "Activated esters allow efficient amidation under mild conditions while remaining compatible with radical polymerization."
     }
}

Paper text: '''

for ind in range(0,len(list_of_files)):
# for ind in range(1):
    file_path=os.path.join(dir_path,list_of_files[ind])
    source = file_path
    converter = DocumentConverter()
    doc = converter.convert(source).document
    md_text = doc.export_to_markdown()
    all_text = " ".join([item.text for item in doc.texts if hasattr(item, "text")])
    doi_pattern = r'10.\d{4,9}/[-._;()/:A-Z0-9]+'
    matches = re.findall(doi_pattern, all_text, re.IGNORECASE)
    doi=matches[0].replace('/','_')
    os.makedirs(os.path.join('./output_test/',doi), exist_ok = True)

    with open(os.path.join(os.path.join('./output_test/',doi),doi+'.md'),'w') as file:
        file.write(md_text)
        
    response = client.responses.create(
        model="gpt-5.2",
        input=prompt+all_text,
    )

    with open(os.path.join(os.path.join('./output_test/',doi),'qa.json'),'w') as file:
        file.write(response.output[0].content[0].text[8:-4])

    with open(os.path.join(os.path.join('./output_test/',doi),'qa.json'), 'r') as json_file:
        data=json.load(json_file)
   
    # add doi and reshuffle answers because ChatGPT likes 'B' more than others
    for entry in data:
        entry['doi']=doi
        entry=shuffle(entry)
    
    with open(os.path.join(os.path.join('./output_test/',doi),'qa.json'), 'w') as json_file:
        json.dump(data,json_file)

    # with open(os.path.join(os.path.join('./output_test/',doi),'qa.jsonl'), 'w') as json_file:
    #     for entry in data:
    #         json.dump(entry,json_file)
    #         json_file.write('\n')